In [ ]:
import tensorflow as tf
import numpy as np
import Utils.BuildMatrix as bm
import numpy as np

In [ ]:
logDirectory = '/home/jemboo/SharedIpynb/graphs/2'
#tensorboard --logdir=/home/jemboo/SharedIpynb/graphs
#fuser 6006/tcp -k

In [ ]:
%%time
import random
mat_sz = 1000
tf.reset_default_graph()
with tf.Session() as sess:
    with tf.device('/gpu:0'): # swap for 'cpu:0' or whatever
        
        a = tf.constant([random.random() for _ in range(mat_sz * mat_sz)], shape=[mat_sz, mat_sz], name='a')
        b = tf.constant([random.random() for _ in range(mat_sz * mat_sz)], shape=[mat_sz, mat_sz], name='b')
        
        f = tf.matmul(a, b)
        for _ in range(100):
            sess.run(f)

#### Block A

In [ ]:
        aP = tf.random_uniform([1000, 1000], name='aP')
        bP = tf.random_uniform([1000, 1000], name='bP')
        a = tf.Variable(aP, name='a')
        b = tf.Variable(bP, name='b')
        sess.run(a.initializer)
        sess.run(b.initializer)

#### Block B

In [ ]:
        a = tf.constant([random.random() for _ in range(mat_sz * mat_sz)], shape=[mat_sz, mat_sz], name='a')
        b = tf.constant([random.random() for _ in range(mat_sz * mat_sz)], shape=[mat_sz, mat_sz], name='b')

In [ ]:
# Block A, CPU: 5.25334620475769
# Block A, GPU: 0.22746562957763672
# Block B, CPU: 0.9498565196990967
# Block B, GPU: 0.9788544178009033

In [12]:
import random
import tensorflow as tf
import numpy as np

In [13]:
logDirectory = '/home/jemboo/SharedIpynb/graphs/B'
walkLength = 200
mat_sz = 500
tf.reset_default_graph()

In [14]:
k = tf.placeholder(tf.float32)
trajects = tf.Variable(tf.zeros(shape=[mat_sz, mat_sz]))
norm = tf.random_normal(shape=[mat_sz, mat_sz], stddev=k)
prod = tf.matmul(norm, norm)
update = tf.assign_add(trajects, prod)
summary_trajects = tf.summary.histogram('trajects', trajects)
summaries = tf.summary.merge_all()

In [15]:
%%time
with tf.device('/cpu:0'):
    with tf.Session() as sess:
        #writer = tf.summary.FileWriter(logDirectory, sess.graph)
        sess.run(trajects.initializer)
        for i in range(walkLength):
            #trn_out, smms = 
            sess.run([update, summaries], feed_dict={k: 1.0 + float(i)/float(walkLength)})
            #writer.add_summary(smms, global_step=i)

CPU times: user 12.6 s, sys: 440 ms, total: 13 s
Wall time: 3.21 s


In [ ]:
CPU times: user 3.94 s, sys: 92 ms, total: 4.03 s
Wall time: 3.45 s

In [ ]:
CPU times: user 3.91 s, sys: 156 ms, total: 4.06 s
Wall time: 3.42 s

In [ ]:
%%time
import random
import tensorflow as tf
import numpy as np

logDirectory = '/home/jemboo/SharedIpynb/graphs/A'
walkLength = 100
d = 2000
tf.reset_default_graph()

#A
trajects = tf.Variable(np.zeros([d, d], dtype=np.float32))
#B
#trajects = tf.Variable(tf.zeros([d, d], dtype=np.float32))
norm = tf.random_normal([d, d], mean=0, stddev=0.04)
update = tf.assign_add(trajects, norm)

summary_trajects = tf.summary.histogram('trajects', trajects)
summaries = tf.summary.merge_all()

with tf.Session() as sess:
    with tf.device('/cpu:0'): # swap for 'cpu:0' or whatever
        writer = tf.summary.FileWriter(logDirectory, sess.graph)

        sess.run(trajects.initializer)

        for i in range(walkLength):
            smms, trn_out = sess.run([summaries, update])
            writer.add_summary(smms, global_step=i)

In [ ]:
# A, CPU: 20.3 s
# A, GPU: 20 s
# B, CPU: 16.7 s
# B, GPU: 16.5 s

In [8]:
tf.reset_default_graph()

In [9]:
dimA = 500
dimB = 500

In [10]:
mAp = tf.random_uniform([dimA, dimB], name='a')
mBp = tf.random_uniform([dimB, dimA], name='b')
mA = tf.Variable(mAp)
mB = tf.Variable(mBp)
mD = tf.matmul(mA, mB)
mE = tf.matrix_inverse(mD)
mF = mB.assign(mE)

In [11]:
%%time
with tf.device("/cpu:0"):
    with tf.Session() as sess:
        #tf.global_variables_initializer().run()
        sess.run(mA.initializer)
        sess.run(mB.initializer)
        for i in range(50):
            prod = sess.run(mF)

CPU times: user 2.83 s, sys: 12 ms, total: 2.84 s
Wall time: 1.36 s


In [ ]:
CPU times: user 532 ms, sys: 556 ms, total: 1.09 s
Wall time: 2.77 s

## map_fn

In [1]:
import random
import tensorflow as tf
import numpy as np
tf.reset_default_graph()

In [2]:
def wank(a):
    return tf.log(a * a)

In [3]:
aP = tf.Variable(tf.random_normal([100, 100], name='aP'))
# squares = tf.map_fn(wank, aP)
squares = tf.map_fn(lambda x: tf.log(tf.abs(tf.sin(x))), aP)
update = tf.assign(aP, squares)

In [4]:
%%time
with tf.device("/gpu:0"):
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        for i in range(2500):
            sess.run(update)

CPU times: user 18.6 s, sys: 2 s, total: 20.6 s
Wall time: 17.2 s


In [ ]:
#from tfCPU
CPU times: user 1min 16s, sys: 19.4 s, total: 1min 36s
Wall time: 22.8 s

In [ ]:
CPU times: user 18.6 s, sys: 2 s, total: 20.6 s
Wall time: 17.2 s

## PDE

In [1]:
import random
import tensorflow as tf
import numpy as np

In [2]:
def make_kernel(a):
  """Transform a 2D array into a convolution kernel"""
  a = np.asarray(a)
  a = a.reshape(list(a.shape) + [1,1])
  return tf.constant(a, dtype=1)

def simple_conv(x, k):
  """A simplified 2D convolution operation"""
  x = tf.expand_dims(tf.expand_dims(x, 0), -1)
  y = tf.nn.depthwise_conv2d(x, k, [1, 1, 1, 1], padding='SAME')
  return y[0, :, :, 0]

def laplace(x):
  """Compute the 2D laplacian of an array"""
  laplace_k = make_kernel([[0.5, 1.0, 0.5],
                           [1.0, -6., 1.0],
                           [0.5, 1.0, 0.5]])
  return simple_conv(x, laplace_k)

In [3]:
tf.reset_default_graph()

In [4]:
N = 1000
# Initial Conditions -- some rain drops hit a pond

# Set everything to zero
u_init = np.zeros([N, N], dtype=np.float32)
ut_init = np.zeros([N, N], dtype=np.float32)

# Some rain drops hit a pond at random points
for n in range(10):
  a,b = np.random.randint(0, N, 2)
  u_init[a,b] = np.random.uniform()

In [5]:
# Parameters:
# eps -- time resolution
# damping -- wave damping
eps = tf.placeholder(tf.float32, shape=())
damping = tf.placeholder(tf.float32, shape=())

# Create variables for simulation state
U  = tf.Variable(u_init)
Ut = tf.Variable(ut_init)

# Discretized PDE update rules
U_ = U + eps * Ut
Ut_ = Ut + eps * (laplace(U) - damping * Ut)

# Operation to update the state
step = tf.group(
  U.assign(U_),
  Ut.assign(Ut_))

In [7]:
%%time
with tf.device("/cpu:0"):
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        for i in range(2500):
            step.run(session=sess, feed_dict={eps: 0.03, damping: 0.04})     
            

InvalidArgumentError: Cannot assign a device for operation 'init_1': Operation was explicitly assigned to /device:GPU:0 but available devices are [ /job:localhost/replica:0/task:0/device:CPU:0 ]. Make sure the device specification refers to a valid device.
	 [[Node: init_1 = NoOp[_device="/device:GPU:0"](^Variable/Assign, ^Variable_1/Assign)]]

Caused by op 'init_1', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/jemboo/tfCPU/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/jemboo/tfCPU/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/jemboo/tfCPU/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/home/jemboo/tfCPU/lib/py

In [ ]:
CPU times: user 6.4 s, sys: 2.9 s, total: 9.3 s
Wall time: 4.24 s

In [ ]:
CPU times: user 5.85 s, sys: 2.82 s, total: 8.68 s
Wall time: 3.67 s